In [ ]:
import pandas as pd

# 读取Excel文件
file_path = 'D:/桌面/CUMCM2024Problems/C题/附件2.xlsx'  # 替换为你的文件路径
sheet1_name = '2023年的农作物种植情况'
sheet2_name = '2023年统计的相关数据'

# 读取Sheet1和Sheet2的数据
df_sheet1 = pd.read_excel(file_path, sheet_name=sheet1_name)
df_sheet2 = pd.read_excel(file_path, sheet_name=sheet2_name)

# 确保两个表中都有'作物编号'列
if '作物编号' not in df_sheet1.columns or '作物编号' not in df_sheet2.columns:
    raise ValueError("'作物编号'列不存在于Sheet1或Sheet2中")

# 合并两个表的数据，根据'作物编号'列进行合并
merged_df = pd.merge(df_sheet1, df_sheet2, on='作物编号', how='inner')

# 计算预期产量
merged_df['预期产量'] = merged_df['种植面积/亩'] * merged_df['亩产量/斤']

# 计算总成本
merged_df['总成本'] = merged_df['种植面积/亩'] * merged_df['种植成本/(元/亩)']

# 选择需要的列
result_df = merged_df[['作物编号', '种植面积/亩', '亩产量/斤', '预期产量', '总成本', '销售单价/(元/斤)', '地块类型']]

# 输出结果到新的Excel文件
output_file_path = 'D:/桌面/CUMCM2024Problems/C题/预期产量.xlsx'
result_df.to_excel(output_file_path, index=False)

print(f"预期产量和总成本已计算并保存到 {output_file_path}")

In [ ]:
import pandas as pd
import pulp

# 读取Excel文件
file_path = 'D:/桌面/CUMCM2024Problems/C题/附件2.xlsx'  # 替换为你的文件路径
sheet1_name = '2023年的农作物种植情况'
sheet2_name = '2023年统计的相关数据'

# 读取Sheet1和Sheet2的数据
df_sheet1 = pd.read_excel(file_path, sheet_name=sheet1_name)
df_sheet2 = pd.read_excel(file_path, sheet_name=sheet2_name)

# 确保两个表中都有'作物编号'列
if '作物编号' not in df_sheet1.columns or '作物编号' not in df_sheet2.columns:
    raise ValueError("'作物编号'列不存在于Sheet1或Sheet2中")

# 合并两个表的数据，根据'作物编号'列进行合并
merged_df = pd.merge(df_sheet1, df_sheet2, on='作物编号', how='inner')

# 定义变量
years = range(2024, 2031)
quarters = range(1, 3)
crops = merged_df['作物编号'].unique()

# 创建线性规划问题
prob = pulp.LpProblem("Crop_Optimization", pulp.LpMaximize)

# 定义决策变量
x = pulp.LpVariable.dicts("x", (crops, years, quarters), lowBound=0, cat='Continuous')

# 定义目标函数
for i in crops:
    for j in years:
        for k in quarters:
            prob += (merged_df[merged_df['作物编号'] == i]['销售单价/(元/斤)'].values[0] * 
                     pulp.lpSum([min(merged_df[merged_df['作物编号'] == i]['预期销售量/斤'].values[0], 
                                     x[i][j][k] * merged_df[merged_df['作物编号'] == i]['亩产量/斤'].values[0]) 
                                 for j in years for k in quarters]) -
                     merged_df[merged_df['作物编号'] == i]['种植成本/(元/亩)'].values[0] * 
                     pulp.lpSum([x[i][j][k] for j in years for k in quarters]))

# 添加约束条件
for j in years:
    for k in quarters:
        prob += pulp.lpSum([x[i][j][k] for i in crops]) <= 1201  # 总耕地面积约束
        for i in crops:
            prob += x[i][j][k] * merged_df[merged_df['作物编号'] == i]['亩产量/斤'].values[0] <= merged_df[merged_df['作物编号'] == i]['预期销售量/斤'].values[0]  # 预期销售量约束

# 求解问题
prob.solve()

# 输出结果
for v in prob.variables():
    print(v.name, "=", v.varValue)

print("Total Revenue = ", pulp.value(prob.objective))

In [38]:
import pandas as pd
import pulp

# 读取Excel文件
file_path1 = 'D:/桌面/CUMCM2024Problems/C题/附件1.xlsx'
file_path2 = 'D:/桌面/CUMCM2024Problems/C题/附件2.xlsx'

# 读取附件1的数据
df_sheet1_1 = pd.read_excel(file_path1, sheet_name='乡村的现有耕地')
df_sheet1_2 = pd.read_excel(file_path1, sheet_name='乡村种植的农作物')

# 读取附件2的数据
df_sheet2_1 = pd.read_excel(file_path2, sheet_name='2023年的农作物种植情况')
df_sheet2_2 = pd.read_excel(file_path2, sheet_name='2023年统计的相关数据')

# 确保两个表中都有'作物编号'列
if '作物编号' not in df_sheet1_2.columns or '作物编号' not in df_sheet2_1.columns or '作物编号' not in df_sheet2_2.columns:
    raise ValueError("'作物编号'列不存在于Sheet1或Sheet2中")

# 合并附件2的数据
merged_df = pd.merge(df_sheet2_1, df_sheet2_2, on=['作物编号', '种植季次'], how='inner')



In [39]:
merged_df

,种植地块,作物编号,作物名称_x,作物类型,种植面积/亩,种植季次,预期销售量/斤,序号,作物名称_y,地块类型,亩产量/斤,种植成本/(元/亩),销售单价/(元/斤)
0,A1,6.0,小麦,粮食,80.0,单季,64000,6,小麦,平旱地,800.0,450.0,3.00-4.00
1,A1,6.0,小麦,粮食,80.0,单季,64000,21,小麦,梯田,760.0,450.0,3.00-4.00
2,A1,6.0,小麦,粮食,80.0,单季,64000,36,小麦,山坡地,720.0,450.0,3.00-4.00
3,A2,7.0,玉米,粮食,55.0,单季,60800,7,玉米,平旱地,1000.0,500.0,2.50-3.50
4,A2,7.0,玉米,粮食,55.0,单季,60800,22,玉米,梯田,950.0,500.0,2.50-3.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,NaN,NaN,NaN,NaN,NaN,NaN,1980,NaN,NaN,NaN,NaN,NaN,NaN
290,NaN,NaN,NaN,NaN,NaN,NaN,1800,NaN,NaN,NaN,NaN,NaN,NaN
291,NaN,NaN,NaN,NaN,NaN,NaN,810,NaN,NaN,NaN,NaN,NaN,NaN
292,NaN,NaN,NaN,NaN,NaN,NaN,990,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
import pandas as pd
import pulp

# 读取Excel文件
file_path1 = 'D:/桌面/CUMCM2024Problems/C题/附件1.xlsx'
file_path2 = 'D:/桌面/CUMCM2024Problems/C题/附件2.xlsx'

# 读取附件1的数据
df_sheet1_1 = pd.read_excel(file_path1, sheet_name='乡村的现有耕地')
df_sheet1_2 = pd.read_excel(file_path1, sheet_name='乡村种植的农作物')

# 读取附件2的数据
df_sheet2_1 = pd.read_excel(file_path2, sheet_name='2023年的农作物种植情况')
df_sheet2_2 = pd.read_excel(file_path2, sheet_name='2023年统计的相关数据')

# 确保两个表中都有'作物编号'列
if '作物编号' not in df_sheet1_2.columns or '作物编号' not in df_sheet2_1.columns or '作物编号' not in df_sheet2_2.columns:
    raise ValueError("'作物编号'列不存在于Sheet1或Sheet2中")

# 合并附件2的数据
merged_df = pd.merge(df_sheet2_1, df_sheet2_2, on=['作物编号', '种植季次'], how='inner')

# 定义变量
years = range(2024, 2031)
quarters = range(1, 3)
crops = merged_df['作物编号'].unique()

# 创建线性规划问题
prob = pulp.LpProblem("Crop_Optimization", pulp.LpMaximize)

# 定义决策变量
x = pulp.LpVariable.dicts("x", (crops, years, quarters), lowBound=0, cat='Continuous')

# 定义目标函数
objective_terms = []
for i in crops:
    for j in years:
        for k in quarters:
            filtered_df = merged_df[(merged_df['作物编号'] == i) & (merged_df['种植季次'] == k)]
            if not filtered_df.empty:
                sales_price = filtered_df['销售单价/(元/斤)'].values[0]
                expected_sales = filtered_df['预期销售量/斤'].values[0]
                yield_per_mu = filtered_df['亩产量/斤'].values[0]
                planting_cost = filtered_df['种植成本/(元/亩)'].values[0]
                
                objective_terms.append((sales_price * min(expected_sales, x[i][j][k] * yield_per_mu) - planting_cost * x[i][j][k]))

prob += pulp.lpSum(objective_terms)

# 添加约束条件
for j in years:
    for k in quarters:
        prob += pulp.lpSum([x[i][j][k] for i in crops]) <= 1201  # 总耕地面积约束
        for i in crops:
            filtered_df = merged_df[(merged_df['作物编号'] == i) & (merged_df['种植季次'] == k)]
            if not filtered_df.empty:
                expected_sales = filtered_df['预期销售量/斤'].values[0]
                yield_per_mu = filtered_df['亩产量/斤'].values[0]
                prob += x[i][j][k] * yield_per_mu <= expected_sales  # 预期销售量约束

# 季节性约束
# 水浇地
for j in years:
    for k in quarters:
        filtered_df = merged_df[(merged_df['种植季次'] == k)]
        if not filtered_df.empty:
            if k == 1:
                prob += pulp.lpSum([x[i][j][k] for i in crops if merged_df[(merged_df['作物编号'] == i) & (merged_df['种植季次'] == k)]['地块类型'].values[0] == '水浇地']) <= 1  # 第一季
            elif k == 2:
                prob += pulp.lpSum([x[i][j][k] for i in crops if merged_df[(merged_df['作物编号'] == i) & (merged_df['种植季次'] == k)]['地块类型'].values[0] == '水浇地']) <= 1  # 第二季
        else:
            print(f"Warning: No data found for quarter {k} in year {j} for 水浇地.")

# 普通大棚
for j in years:
    for k in quarters:
        filtered_df = merged_df[(merged_df['种植季次'] == k)]
        if not filtered_df.empty:
            if k == 1:
                prob += pulp.lpSum([x[i][j][k] for i in crops if merged_df[(merged_df['作物编号'] == i) & (merged_df['种植季次'] == k)]['地块类型'].values[0] == '普通大棚']) <= 1  # 第一季
            elif k == 2:
                prob += pulp.lpSum([x[i][j][k] for i in crops if merged_df[(merged_df['作物编号'] == i) & (merged_df['种植季次'] == k)]['地块类型'].values[0] == '普通大棚']) <= 1  # 第二季
        else:
            print(f"Warning: No data found for quarter {k} in year {j} for 普通大棚.")

# 智慧大棚
for j in years:
    for k in quarters:
        filtered_df = merged_df[(merged_df['种植季次'] == k)]
        if not filtered_df.empty:
            if k == 1:
                prob += pulp.lpSum([x[i][j][k] for i in crops if merged_df[(merged_df['作物编号'] == i) & (merged_df['种植季次'] == k)]['地块类型'].values[0] == '智慧大棚']) <= 1  # 第一季
            elif k == 2:
                prob += pulp.lpSum([x[i][j][k] for i in crops if merged_df[(merged_df['作物编号'] == i) & (merged_df['种植季次'] == k)]['地块类型'].values[0] == '智慧大棚']) <= 1  # 第二季
        else:
            print(f"Warning: No data found for quarter {k} in year {j} for 智慧大棚.")

# 求解问题
prob.solve()

# 输出结果
results = []
for v in prob.variables():
    results.append([v.name, v.varValue])

# 将结果保存到Excel文件
results_df = pd.DataFrame(results, columns=['Variable', 'Value'])
results_df.to_excel('D:/桌面/CUMCM2024Problems/C题/预期产量.xlsx', index=False)

print("Total Revenue = ", pulp.value(prob.objective))

Total Revenue =  None
